In [9]:
from queue import PriorityQueue
from copy import deepcopy

SIDE = 3
puzzleSize = SIDE * SIDE   # 8-puzzle is 3 x 3

'''
The search node class
- state is a list representing the puzzle arrangement in row-major order. 
   the 0 represents blank. ex> [1, 2, 3, 8, 0, 4, 7, 6, 5]
- parent is a parent node
- operator is one of U, D, L, R
- evaluation value of node f = g + h
- moves record the path (sequence of actions)
'''
class Node:
    def __init__(self, state, parent, operator, g, h):
        self.state = state
        self.parent = parent
        self.operator = operator
        self.g = g
        self.h = h
        self.f = self.g + self.h
        if parent is None:
            self.moves = operator
        else:
            self.moves = parent.moves + '-' + operator
    def __lt__(self, other):
        return self.f < other.f

'''
ASTAR performs A* search
- OPEN is a PriorityQueue of (node.f, node).
- CLOSED is a list of states
- When a next state is in OPEN, it should be updated.
   But instead, here we just put it into OPEN,
   and when it is chosen, we ignore the state if it is in CLOSED
xpand returns list of next (child) nodes
'''
def ASTAR(startNode):
    OPEN = PriorityQueue()
    OPEN.put((startNode.f, startNode))
    CLOSED = []
    while not(OPEN.empty()):
        CurrentNode = OPEN.get()[1] # OPEN에 저장된 가장 위에 있는 큐의 원소 중 Node를 가져옴 0 - 휴리스틱 값, 1 - 노드
        
        if CurrentNode.state not in CLOSED:  # 큐에서 가장 위에 있는 원소가 CLOSED에 없을 때
            if goaltest(CurrentNode.state): # 만약 현재 노드가 골 노드이면 while을 멈춤
                    return CurrentNode  # 현재노드를 리턴
            else: # 큐에서 가장 위에 있는 원소가 CLOSED에 있을 때
                CLOSED.append(CurrentNode.state) # 현재 노드를 CLOSED에 넣기
                ChildNodeList = expand(CurrentNode) # 근접노드를 노드리스트에 넣기
                for Node in ChildNodeList: # 노드리스트에 있는 모든 노드를 돌기
                    if Node.state not in CLOSED: # 노드가 CLOSED에 없다면 OPEN에 넣기
                        OPEN.put((Node.f, Node))
                        
        print(".", end='') # 한 번 탐색을 진행한 후 .을 찍음

        
    return None # wihle 문이 끝날 때까지 리턴되지 못하면 골을 찾을 수 없는 것이기에 none을 리턴
                
'''
goaltest returns 1 if state is same as the goalState
'''
def goaltest(state):
    if state == goalState:
        return 1
    else:
        return 0

'''
heuristic returns estimated distant from state to the goalState
'''
def heuristic(state):
    tiles = 0  # 정답과 다른 타일의 갯수를 세기 위한 변수
    for i in range (puzzleSize): # 모든 퍼즐을 순회
         if (state[i] != 0): # 0 (공백) 아니고
            if(state[i] != goalState[i]): # 정답과 다르면
                tiles += 1  # tiles + 1
            
    return tiles # 리턴

'''
expand returns list of next (child) nodes
'''
def expand(node):
    nodeList = []
    state = node.state
    blank = state.index(0)   # location of blank
    row = int(blank / SIDE)
    col = blank % SIDE
    
    # up
    if not (row == 0):
        childState = deepcopy(state)
        childState[blank] = childState[blank-SIDE]
        childState[blank-SIDE] = 0
        nodeList.append(Node(childState, node, "U", node.g+1, heuristic(childState)))
    # down
    if not (row == SIDE-1):
        childState = deepcopy(state)
        childState[blank] = childState[blank+SIDE]
        childState[blank+SIDE] = 0
        nodeList.append(Node(childState, node, "D", node.g+1, heuristic(childState)))
    # left
    if not (col == 0):
        childState = deepcopy(state)
        childState[blank] = childState[blank-1]
        childState[blank-1] = 0
        nodeList.append(Node(childState, node, "L", node.g+1, heuristic(childState)))
    # right
    if not (col == SIDE-1):
        childState = deepcopy(state)
        childState[blank] = childState[blank+1]
        childState[blank+1] = 0
        nodeList.append(Node(childState, node, "R", node.g+1, heuristic(childState)))

    return nodeList

'''
printState print 3x3 puzzle state
'''
def printState(state):
    for i in range(3):
        for j in range(3):
            print(state[i*3 + j], end="\t")
        print("")
    print("")
    
'''
printSolutionPath show solution path recursively
'''
def printSolutionPath(node):
    if (node == None): return
    printSolutionPath(node.parent)
    printState(node.state)


In [10]:
'''
The start and goal states
'''
startState = [1, 2, 3, 
              8, 0, 4, 
              7, 6, 5]
goalState = [7, 8, 1,
             6, 0, 2,
             5, 4, 3]

print("Start state")
printState(startState)
print("Goal state")
printState(goalState)

'''
Make start node and do A* search
'''
startNode = Node(startState, None, "Start", 0, heuristic(startState))
print("Start A* search")
goalNode = ASTAR(startNode)
print(" Finished")

'''
If succeeded, show the solution path
'''
if (goalNode == None):
    print("A* Failed ")
else:
    print("A* Succeeded ")
    print("solution path: ", goalNode.moves)
    printSolutionPath(goalNode)


Start state
1	2	3	
8	0	4	
7	6	5	

Goal state
7	8	1	
6	0	2	
5	4	3	

Start A* search
........................................................................................................................................................................................................................................................................................................................................................ Finished
A* Succeeded 
solution path:  Start-U-L-D-D-R-R-U-U-L-L-D-D-R-R-U-L
1	2	3	
8	0	4	
7	6	5	

1	0	3	
8	2	4	
7	6	5	

0	1	3	
8	2	4	
7	6	5	

8	1	3	
0	2	4	
7	6	5	

8	1	3	
7	2	4	
0	6	5	

8	1	3	
7	2	4	
6	0	5	

8	1	3	
7	2	4	
6	5	0	

8	1	3	
7	2	0	
6	5	4	

8	1	0	
7	2	3	
6	5	4	

8	0	1	
7	2	3	
6	5	4	

0	8	1	
7	2	3	
6	5	4	

7	8	1	
0	2	3	
6	5	4	

7	8	1	
6	2	3	
0	5	4	

7	8	1	
6	2	3	
5	0	4	

7	8	1	
6	2	3	
5	4	0	

7	8	1	
6	2	0	
5	4	3	

7	8	1	
6	0	2	
5	4	3	

